<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Depolarization-of-opinions" data-toc-modified-id="Depolarization-of-opinions-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Depolarization of opinions</a></span><ul class="toc-item"><li><span><a href="#Colab-Setup" data-toc-modified-id="Colab-Setup-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Colab Setup</a></span></li><li><span><a href="#Imports-and-settings" data-toc-modified-id="Imports-and-settings-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Imports and settings</a></span></li></ul></li><li><span><a href="#Example-usage" data-toc-modified-id="Example-usage-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Example usage</a></span><ul class="toc-item"><li><span><a href="#Parameters" data-toc-modified-id="Parameters-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Parameters</a></span></li><li><span><a href="#Create-EchoChamber" data-toc-modified-id="Create-EchoChamber-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Create EchoChamber</a></span></li><li><span><a href="#Set-activity-based-on-powerlaw-distribution" data-toc-modified-id="Set-activity-based-on-powerlaw-distribution-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Set activity based on powerlaw distribution</a></span></li><li><span><a href="#Connection-probabilities" data-toc-modified-id="Connection-probabilities-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>Connection probabilities</a></span></li><li><span><a href="#Social-Interactions" data-toc-modified-id="Social-Interactions-2.5"><span class="toc-item-num">2.5&nbsp;&nbsp;</span>Social Interactions</a></span></li><li><span><a href="#Dynamics" data-toc-modified-id="Dynamics-2.6"><span class="toc-item-num">2.6&nbsp;&nbsp;</span>Dynamics</a></span></li><li><span><a href="#Run-and-plot-results" data-toc-modified-id="Run-and-plot-results-2.7"><span class="toc-item-num">2.7&nbsp;&nbsp;</span>Run and plot results</a></span></li><li><span><a href="#Save/Load" data-toc-modified-id="Save/Load-2.8"><span class="toc-item-num">2.8&nbsp;&nbsp;</span>Save/Load</a></span></li></ul></li><li><span><a href="#Example-results" data-toc-modified-id="Example-results-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Example results</a></span><ul class="toc-item"><li><span><a href="#Neutral-opinion" data-toc-modified-id="Neutral-opinion-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Neutral opinion</a></span></li><li><span><a href="#Radicalisation-of-opinions" data-toc-modified-id="Radicalisation-of-opinions-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Radicalisation of opinions</a></span></li><li><span><a href="#Polarisation-of-opinions" data-toc-modified-id="Polarisation-of-opinions-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Polarisation of opinions</a></span></li><li><span><a href="#Compare-opinons" data-toc-modified-id="Compare-opinons-3.4"><span class="toc-item-num">3.4&nbsp;&nbsp;</span>Compare opinons</a></span></li></ul></li><li><span><a href="#Plot-types" data-toc-modified-id="Plot-types-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Plot types</a></span><ul class="toc-item"><li><span><a href="#$K-\alpha$-phase-space" data-toc-modified-id="$K-\alpha$-phase-space-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>$K-\alpha$ phase space</a></span></li><li><span><a href="#Activity-vs-Opinion" data-toc-modified-id="Activity-vs-Opinion-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Activity vs Opinion</a></span></li><li><span><a href="#Show-adjacency-matrix" data-toc-modified-id="Show-adjacency-matrix-4.3"><span class="toc-item-num">4.3&nbsp;&nbsp;</span>Show adjacency matrix</a></span></li><li><span><a href="#Agent-opinons-at-$t$" data-toc-modified-id="Agent-opinons-at-$t$-4.4"><span class="toc-item-num">4.4&nbsp;&nbsp;</span>Agent opinons at $t$</a></span></li><li><span><a href="#Nearest-Neighbour" data-toc-modified-id="Nearest-Neighbour-4.5"><span class="toc-item-num">4.5&nbsp;&nbsp;</span>Nearest Neighbour</a></span></li></ul></li><li><span><a href="#Noisy-Echo-Chamber" data-toc-modified-id="Noisy-Echo-Chamber-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Noisy Echo Chamber</a></span><ul class="toc-item"><li><span><a href="#Check-dt-with-noise" data-toc-modified-id="Check-dt-with-noise-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>Check <code>dt</code> with noise</a></span></li><li><span><a href="#Params-vs-noise" data-toc-modified-id="Params-vs-noise-5.2"><span class="toc-item-num">5.2&nbsp;&nbsp;</span>Params vs noise</a></span><ul class="toc-item"><li><span><a href="#$T_D-=-0$" data-toc-modified-id="$T_D-=-0$-5.2.1"><span class="toc-item-num">5.2.1&nbsp;&nbsp;</span>$T_D = 0$</a></span></li><li><span><a href="#$T_D-=-10$" data-toc-modified-id="$T_D-=-10$-5.2.2"><span class="toc-item-num">5.2.2&nbsp;&nbsp;</span>$T_D = 10$</a></span></li></ul></li><li><span><a href="#Delayed-noise" data-toc-modified-id="Delayed-noise-5.3"><span class="toc-item-num">5.3&nbsp;&nbsp;</span>Delayed noise</a></span></li><li><span><a href="#TODO:-Using-Julia" data-toc-modified-id="TODO:-Using-Julia-5.4"><span class="toc-item-num">5.4&nbsp;&nbsp;</span>TODO: Using Julia</a></span><ul class="toc-item"><li><span><a href="#Julia-Issues" data-toc-modified-id="Julia-Issues-5.4.1"><span class="toc-item-num">5.4.1&nbsp;&nbsp;</span>Julia Issues</a></span></li></ul></li></ul></li></ul></div>

# Depolarization of opinions

Let us consider a system of $N$ agents, each agent $i$
characterized by a dynamic opinion variable $x_i(t)$. For
the sake of simplicity, we consider opinions to be one dimensional,
with $x_i \in [-\infty;+\infty]$. The sign of the opinion
$x_i$, $\sigma(x_i)$, describes the agent's qualitative stance towards
 a binary issue of choice, such as the preference
between two candidates or a pro/con attitude in a controversial
topic. The absolute value of $x_i$, $|x_i|$, describes
the opinion's strength, or conviction, with respect to one
of the sides: the larger $|x_i|$, the more extreme the opinion
of agent $i$.

## Colab Setup

In [ ]:
try:
    # connect GDrive for retrieving/saving results
    from google.colab import drive
    drive.mount('/content/drive')

    # Clone github repository
    from os.path import join  
    GIT_USERNAME = "chriscurrin"
    # TODO: remove this from github before publishing
    GIT_TOKEN = "bbf24b4521d81a3893d6c93eb806d41b47322a33"  
    GIT_REPOSITORY = "opinion_dynamics" 
    GIT_PATH = "https://" + GIT_TOKEN + "@github.com/" + GIT_USERNAME + "/" + GIT_REPOSITORY + ".git"
    !rm -rf ./temp
    !git clone "{GIT_PATH}" ./temp

    # add to path
    import sys
    sys.path.append('./temp')
    # need latest tqdm version for tenumerate
    !pip install --upgrade tqdm
    # create symlink between a Drive folder and the cache for persistence between sessions
    !ln -s "/content/drive/My Drive/Colab Notebooks/opdynamics/.cache" ".cache"
    !ln -s "/content/drive/My Drive/Colab Notebooks/opdynamics/output" "output"
except ModuleNotFoundError:
    print("local notebook")
    pass

## Imports and settings

In [ ]:
%reload_ext autoreload
%autoreload 2

import logging
import itertools
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import seaborn as sns

from tqdm.notebook import tqdm
from matplotlib.ticker import MaxNLocator
from matplotlib.colors import LogNorm

from opdynamics.utils.distributions import negpowerlaw
from opdynamics.utils.plot_utils import move_cbar_label_to_title
try:
    import opdynamics.simulation as Simulation
except AttributeError:
    print("restart runtime")
from opdynamics.dynamics.echochamber import EchoChamber, NoisyEchoChamber, logger as eclogger
from opdynamics.visualise.visechochamber import VisEchoChamber
from opdynamics.visualise.dense import show_K_alpha_phase, show_activity_vs_opinion

from opdynamics.utils.constants import *

try:
    os.makedirs("output")
except FileExistsError:
    pass

np.random.seed(1337)
sns.set_context("notebook")
logging.basicConfig(level=logging.DEBUG)
logging.getLogger().setLevel(logging.DEBUG)
logging.getLogger('matplotlib').setLevel(logging.INFO)

# Example usage

## Parameters

In [ ]:
num_agents = 1000
m = 10  # number of other agents to interact with
alpha = 3  # controversialness of issue (sigmoidal shape)
K = 3  # social interaction strength
epsilon = 1e-2  # minimum activity level with another agent
gamma = 2.1  # power law distribution param
beta = 3  # power law decay of connection probability
r = 0.5 # probability of a mutual interaction
activity_distribution = negpowerlaw

## Create EchoChamber

In [ ]:
ec = EchoChamber(num_agents, m, K, alpha)
vis = VisEchoChamber(ec)

## Set activity based on powerlaw distribution

$$ F(a) = \frac{1 - γ}{1 - ε^{1 - γ}} a^{-\gamma} \tag{1}$$

In [ ]:
ec.set_activities(activity_distribution, gamma, epsilon)
fig, ax = plt.subplots(1,2, figsize=(8,5))
fig.subplots_adjust(wspace=0.3)
fig.suptitle("t=0")
vis.show_activities(ax=ax[0])
vis.show_activity_vs_opinion(ax=ax[1])
sns.despine()

## Connection probabilities

$$ p_{ij} = \frac{|x_i - x_j|^{-\beta}}{\sum_j |x_i - x_j|^{-\beta}} \tag{2}$$

In [ ]:
ec.set_connection_probabilities(beta=beta)
fig, ax = vis.show_connection_probabilities("mesh", sort=True, 
                                  cmap=sns.cubehelix_palette(light=1, as_cmap=True), 
                                  cbar_kws=dict(pad=0.05))
# move label to title
move_cbar_label_to_title(fig)

## Social Interactions
- adjacency matrix $A_{ij}(t)$ can be calculated before numerical integration (`lazy=False`) or at time $t$ (`lazy=True`).
- Cumulative adjacency matrix $\sum_{t=0}^T A_{ij}(t)$ can be plotted a number of ways using `vis.show_adjacency_matrix(...)`
    - `clustermap`
    - `heatmap`
    - `mesh`
    
    Interactions can be sorted using `sort=True` for plotting to identify clusters

In [ ]:
ec.set_social_interactions(0.5, lazy=False, t_end=5, dt=0.01)
vis.show_adjacency_matrix("mesh", True)

## Dynamics


Assuming that the opinion dynamics is solely
driven by the interactions among agents, we formulate
the model as $N$ coupled ordinary differential equations,

$$ \dot{x_i} = -x_i + K \sum_{j=1}^{N} A_{ij}(t) \tanh(\alpha x_j) \tag{3}$$

where K > 0 denotes the social interaction strength
among agents and $\alpha$ determines the sigmoidal shape of
the hyperbolic tangent. The opinion of an agent $i$ follows
the aggregated social input from the set of his/her neighbors
at time $t$, determined by the symmetric adjacency
matrix of the temporal network $A_{ij}(t)$, where $A_{ij}(t) = 1$
if agents $i$ and $j$ are interacting at time $t$, $A_{ij}(t) = 0$
otherwise. A similar model with static connectivity has
previously been used to describe the dynamics of neural
networks showing a transition from stationary to chaotic
phase [30].
The parameter $\alpha > 0$ tunes the degree of non-linearity
between an agent's opinion and the social influence s/he exerts on others.

In [ ]:
ec.set_dynamics()

## Run and plot results

In [ ]:
dt = 0.01
T = 0.5
ec.run_network(dt=dt, t_end=T, method='Euler')

In [ ]:
# some options on how to plot
fig, ax = plt.subplots(nrows=3, sharex='col', sharey='col', figsize=(6,6))
fig.subplots_adjust(hspace=0.05)
vis.show_opinions(color_code=True, ax=ax[0], title=True)    # markers
vis.show_opinions(color_code='line', ax=ax[1], title=False)  # lines (slowest)
vis.show_opinions(color_code=False, ax=ax[2], title=False)   # agents uniquely coloured
sns.despine(fig)

## Save/Load

Given the same parameters, a network can be restored without explicitly calling `run_network`. 

A simulation is saved using `ec.save(only_last=True)`. If `only_last` is `False`, the entire duration of the simulation is saved.

If a simulation exists for the network parameters, `ec.load()` restores the saved states and returns `True` (returns `False` otherwise).

In [ ]:
filename = ec.save()
new_ec = EchoChamber(num_agents, m, K, alpha)
new_ec.set_activities(activity_distribution, gamma, epsilon)
new_ec.set_connection_probabilities(beta=beta)
new_ec.set_social_interactions(0.5, lazy=True) # note only the `r` is the important parameter
new_ec.set_dynamics()
new_ec.load(dt, T)

# Example results

> The convenient `Simulation.run_params` static method is used.

In [ ]:
# General params

N=1000
m=10
T=10
epsilon=1e-2
gamma=2.1
r=0.5 # probability of mutual interaction
dt=0.01

# Specific params for different dynamics
param_set={
    "neutral": dict(K=3,alpha=0.05,beta=2),
    "radical": dict(K=3,alpha=3,beta=0),
    "polar": dict(K=3,alpha=3,beta=3)
    }

## Neutral opinion

- $K = 3$
- $\alpha = 0.05$
- $beta = 2$

In [ ]:
ec_neutral = Simulation.run_params(EchoChamber, N=N, m=m, **param_set['neutral'],
                                   activity=activity_distribution, epsilon=epsilon, gamma=gamma, r=r,
                                   dt=dt, T=T, plot_opinion=True,
                                   cache="all")
sns.despine()

## Radicalisation of opinions

- $K = 3$
- $\alpha = 3$
- $beta = 0$

In [ ]:
ec_radical = Simulation.run_params(EchoChamber, N=N, m=m, **param_set['radical'],
                                   activity=activity_distribution, epsilon=epsilon, gamma=gamma, r=r,
                                   dt=dt, T=T, plot_opinion=True,
                                   cache="all")
sns.despine()

## Polarisation of opinions

- $K = 3$
- $\alpha = 3$
- $beta = 3$

In [ ]:
ec_polar = Simulation.run_params(EchoChamber, N=N, m=m, **param_set['polar'],
                                 activity=activity_distribution, epsilon=epsilon, gamma=gamma, r=r,
                                 dt=dt, T=T, plot_opinion=True,
                                 cache="all")
sns.despine()

## Compare opinons

In [ ]:
fig = plt.figure(constrained_layout=True)
spec = gridspec.GridSpec(nrows=2, ncols=2, figure=fig)
f_ax1 = fig.add_subplot(spec[0, 0]) # top left
f_ax2 = fig.add_subplot(spec[1, 0]) # bottom left
f_ax3 = fig.add_subplot(spec[:, 1]) # right column

VisEchoChamber(ec_neutral).show_opinions(ax=f_ax1, title="Neutral")
VisEchoChamber(ec_radical).show_opinions(ax=f_ax2, title="Radicalised")
VisEchoChamber(ec_polar).show_opinions(ax=f_ax3, title="Polarised")
sns.despine()

# Plot types

## $K-\alpha$ phase space

- $K \in [0,4]$
- $\alpha \in [0,4]$

with
- $beta = 0.5$
- $r = 0.5$


In [ ]:
import os
import gc
eclogger.setLevel(logging.DEBUG)

beta = 0.5
r = 0.5


# how many simulations to run
num_states = 8
start = 0
stop = 4
K_range = np.round(np.linspace(start,stop,num_states), 2)
alpha_range = np.round(np.linspace(start,stop,num_states), 2)


# where to save the file
file_name = ".cache/K-alpha-phase.h5"

# the efficient HDF format is used for saving and loading DataFrames.
if os.path.exists(file_name):
    df = pd.read_hdf(file_name)
else:
    df = pd.DataFrame(index=K_range, columns=alpha_range, dtype=float)
    for K,alpha in tqdm(list(itertools.product(K_range,alpha_range))):
        ec = Simulation.run_params(EchoChamber,N=N, m=m, K=K, alpha=alpha, beta=beta, 
                                   activity=activity_distribution, epsilon=epsilon, gamma=gamma, r=r,
                                   dt=dt, T=T, 
                                   lazy=True, 
                                   cache=False, # don't cache full time of sim
                                   plot_opinion=False)
        t, mu = ec.get_mean_opinion(-1)
        df.loc[K, alpha] = mu
        # clear some memory
        del ec
        gc.collect()
    df.to_hdf(file_name, key='df')

In [ ]:
show_K_alpha_phase(df)
fig = plt.gcf()
fig.savefig("output/k-alpha-phase.pdf")

## Activity vs Opinion
Polarised param set
- $r = 0.65$

> to reproduce the density in the papers, $10^3$ "opinion states" need to be plotted, with each opinion state consisting of 1000 agents' opinions.

In [ ]:
# suppress logging for clean progressbar
logging_level = logging.getLogger().getEffectiveLevel()
logging.getLogger().setLevel(logging.WARNING)

# parameters
N=1000
m=10
T=5
activity_distribution=negpowerlaw
epsilon=1e-2
gamma=2.1
dt=0.01
r=0.65

# how many simulations to run
try:
    from google.colab import drive
    num_states = 1000
except ModuleNotFoundError:
    num_states = 10
# where to save the file
file_name = ".cache/activity_vs_opinion.h5"

# helper array
agent_indices = np.arange(N, dtype=int)

# the efficient HDF format is used for saving and loading DataFrames.
with pd.HDFStore(file_name) as hdf:
    # get keys from the store to know how many simulations have already been saved
    states = hdf.keys()
    states_completed = len(states)
    # we start iterating from states_completed as the `seed` parameter uses `i`
    for i in tqdm(range(states_completed, num_states)):
        ec = Simulation.run_params(EchoChamber,N=N, m=m, **param_set['polar'],
                                   activity=activity_distribution, epsilon=epsilon, gamma=gamma, r=r,
                                   dt=dt, T=T, lazy=True, 
                                   cache=False, # don't cache time portion
                                   seed=i)
        # create dataframe for this iteration
        df = pd.DataFrame({"activity": ec.activities,
                           "opinion": ec.opinions
                          })
        # save to disk for use later
        hdf.append(f"d{i}", df)

# return to previous logging (more or less - we change root level)
logging.getLogger().setLevel(logging_level)


In [ ]:
# collect all data into a dataframe
with pd.HDFStore(file_name) as hdf:
    df_ao = pd.concat([hdf.get(key) for key in hdf.keys()])
df_ao.shape

In [ ]:
# plot everything
# (very good idea to rasterize for saving)
fig, ax = show_activity_vs_opinion(df_ao.opinion.values, df_ao.activity.values,
                                   bins=N, norm=LogNorm(vmin=1/N),
                                   cbar_kws=dict(cax=True),
                                   s=0.1, rasterized=True)
ax.set_ylim(0, 0.5)
sns.despine()
fig.savefig("output/activity_vs_opinion.pdf", dpi=500)

In [ ]:
# clear up some ram
del df_ao

## Show adjacency matrix

In [ ]:
vis = VisEchoChamber(ec_polar)
vis.show_adjacency_matrix("clustermap", ax=False, fig=False, sort=False)
fig, ax = plt.subplots(2, 1, figsize=(10,20))
vis.show_adjacency_matrix("heatmap", sort=True, square=True, ax=ax[0])
vis.show_adjacency_matrix("mesh", sort=True, ax=ax[1])


## Agent opinons at $t$

In [ ]:
fig, ax = plt.subplots(2, 1, figsize=(10,10), sharex=True)
fig.subplots_adjust(hspace=0.3)
vis.show_agent_opinions(direction=True, sort=True, ax=ax[0])
vis.show_opinions_snapshot(ax=ax[1], kde_kws=dict(shade=True))
ax[1].set_xticklabels(ax[1].get_xticks())
sns.despine()

In [ ]:
fig, ax = plt.subplots(1, 2, gridspec_kw=dict(width_ratios=[10,1]))
fig.subplots_adjust(wspace=0.1)
vis.show_opinions(ax=ax[0], title="Polarised")
vis.show_opinions_snapshot(ax=ax[1], kde_kws=dict(shade=False, bw=0.15), vertical=True)
sns.despine()
sns.despine(ax=ax[1], left=True)
ax[1].set_ylabel("")
ax[1].tick_params(axis='y', left=False, labelleft=False)

## Nearest Neighbour

In [ ]:
g = vis.show_nearest_neighbour(bw=0.5)
# adjust limits based on xaxis
from scipy import stats
x_data, y_data = g.ax_marg_x.get_lines()[0].get_data()
s = stats.describe(y_data)
low_bound = np.max(s.mean-s.variance, 0)
mask = (low_bound < y_data)
lim = x_data[mask][0]*1.5, x_data[mask][-1]*1.5
g.ax_joint.set_xlim(*lim)
# keep it 'square' (xlim == ylim)
g.ax_joint.set_ylim(*lim)
g.savefig("output/nearest_neighbour.pdf")

In [ ]:
# try "paper" "poster" "talk" or "notebook"
with sns.plotting_context("notebook"):
    fig, ax = vis.show_summary(fig_kwargs=dict(figsize=(12,12), constrained_layout=False))
fig.subplots_adjust(wspace=0.5, hspace=0.5)
sns.despine()

# Noisy Echo Chamber

In [ ]:
# suppress logging for clean progressbar
logging_level = logging.getLogger().getEffectiveLevel()
logging.getLogger().setLevel(logging.WARNING)
kwargs = dict(N=1000,
              m=10,
              T=5,
              epsilon=1e-2,
              gamma=2.1,
              dt=0.005,
              K=2,
              beta=1,
              alpha=3,
              r=0.65,
             )
D_range = [0, 0.01, 0.1, 0.5, 1, 10]
nec_arr = Simulation.run_noise_range(D_range, plot_opinion=True, **kwargs)
fig = plt.gcf()
fig.subplots_adjust(hspace=-0.5)
for ax in fig.axes:
    ax.set_facecolor("None")
    sns.despine(ax=ax, bottom=True, left=True)
    ax.tick_params(bottom=False, left=False, labelleft=False)
fig.savefig("output/noise.pdf")
logging.getLogger().setLevel(logging_level)

## Check `dt` with noise

In [ ]:
# suppress logging for clean progressbar
logging_level = logging.getLogger().getEffectiveLevel()
logging.getLogger().setLevel(logging.WARNING)
kwargs = dict(N=1000,
              m=10,
              T=1,
              epsilon=1e-2,
              gamma=2.1,
              dt=0.001,
              K=2,
              beta=1,
              alpha=3,
              r=0.65,
             )
D_range = [0, 0.01, 0.1, 1, 10]
dt_range = np.arange(0.001, 0.01+0.001, 0.001)
Simulation.run_noise_other_range(D_range, "dt", dt_range,
                                 plot_opinion=True, title="dt",
                                 label_precision=3, **kwargs)
fig = plt.gcf()
fig.savefig("output/dt.pdf")
logging.getLogger().setLevel(logging_level)

## Params vs noise

In [ ]:
kwargs = dict(N=1000,
              m=10,
              T=10,
              epsilon=1e-2,
              gamma=2.1,
              dt=0.01,
              K=2,
              beta=1,
              alpha=3,
              r=0.65,
             )
D_range = [0, 0.01, 0.1, 0.5, 1.0]

other_vars = {
    'beta':  {
        'range': [0, 1, 2, 3],
        'title': "$\\beta$"
    },
    'alpha': {
        'range': [0.001, 1, 2, 3], 
        'title': "$\\alpha$"
    },
}

### $T_D = 0$

In [ ]:
# suppress logging for clean progressbar
logging_level = logging.getLogger().getEffectiveLevel()
logging.getLogger().setLevel(logging.WARNING)

for key, other in other_vars.items():
    print(f"running noise vs {key}")
    Simulation.run_noise_other_range(D_range, key, other['range'],
                                     plot_opinion=True, title=other['title'],
                                     subplot_kws=dict(sharex='all', sharey='col'),
                                     **kwargs)
    fig = plt.gcf()
    fig.savefig(f"output/{key}.pdf")

logging.getLogger().setLevel(logging_level)

### $T_D = 10$

In [ ]:
# suppress logging for clean progressbar
logging_level = logging.getLogger().getEffectiveLevel()
logging.getLogger().setLevel(logging.WARNING)
# equal time no-noise and noise
noise_start = kwargs['T']
for key, other in other_vars.items():
    print(f"running noise at T{noise_start} vs {key}")
    Simulation.run_noise_other_range(D_range, key, other['range'],
                                     noise_start=noise_start, 
                                     plot_opinion=True, title=f"{other['title']} $T_D={noise_start}$",
                                     subplot_kws=dict(sharex='all', sharey='col'),
                                     **kwargs)
    fig = plt.gcf()
    fig.savefig(f"output/{key}_T{noise_start}.pdf")

logging.getLogger().setLevel(logging_level)

## Delayed noise

In [ ]:
# suppress logging for clean progressbar
logging_level = logging.getLogger().getEffectiveLevel()
logging.getLogger().setLevel(logging.WARNING)
kwargs = dict(N=1000,
              m=10,
              activity_distribution = negpowerlaw,
              epsilon=1e-2,
              gamma=2.1,
              dt=0.01,
              K=3,
              beta=3,
              alpha=3,
              r=0.65           
             )
D=0.1

noise_start = 10.
noise_length = 10.
recovery = 10.
num = 1
interval = 0
nec = Simulation.run_periodic_noise(noise_start, noise_length, recovery, interval=interval, num=num, 
                                    **kwargs,
                                    D=D, plot_opinion=True)
fig = plt.gcf()
fig.savefig("output/delayed_noise.pdf")
logging.getLogger().setLevel(logging_level)

In [ ]:
from tqdm.notebook import tqdm
# suppress logging for clean progressbar
logging_level = logging.getLogger().getEffectiveLevel()
logging.getLogger().setLevel(logging.WARNING)
eclogger.setLevel(logging.WARNING)

noise_start = 5.
noise_length = 10.
recovery = 10.
for D in tqdm([0.1, 0.5, 1], desc='noise'):
    for num in tqdm([1, 2, 5], desc='num'):
        intervals = [0.1, 0.5, 1, 2] if num>1 else [0]
        for interval in tqdm(intervals, desc='interval'):
            nec = Simulation.run_periodic_noise(noise_start, noise_length, recovery, interval=interval, num=num, 
                                                **kwargs,
                                                D=D, plot_opinion=True)
            fig = plt.gcf()
            fig.suptitle(f"D={D}, num={num}, interval={interval}")
            fig.savefig(f"output/noise_D{D}_{num}-{interval}.pdf", dpi=500)
logging.getLogger().setLevel(logging_level)

## TODO: Using Julia

`DiffentialEquations.jl`

Setup:

1. Download Julia and add `julia` to system path.
2. `pip install diffeqpy numba`

3. Install Python compatibility
    ```python
   import julia
   julia.install()
   import diffeqpy
   diffeqpy.install()
   ```
4. Test it works
   ```python
   from diffeqpy import de
   ```

### Julia Issues

Need to re-specify equation because of element-wise multiplication.
